In [37]:
from collections import Counter
import pandas as pd
import numpy as np

In [2]:
file_name = "9M-5-[1-2001]-[a]-lcm.out"

# Product adoption & Loyalty

In [3]:
links = pd.read_csv(f"../../plots/links/{file_name}")
links = links.drop(["user_id"],axis=1)
groups = pd.read_csv(f"../../plots/groups/{file_name}",index_col=0)
groups = groups.drop(['depth',"a","size","support",],axis=1)
df = links.merge(groups.reset_index(),left_on="source",right_on='index').merge(groups.reset_index(),left_on="target",right_on='index')

In [11]:
output_file = f"queries_results/python_{file_name.replace('out','csv')}"

In [12]:
before_promo_period = groups.period.min()
after_promo_period = groups.period.max()
promo_period = list(set(groups.period.unique()).difference((before_promo_period,after_promo_period)))[0]

promo_items = pd.read_csv("../../datasets/Total/items.csv")
promo_list_items = promo_items.ARTICLE_ID.tolist()

def promotion_query(x,query_name):
    if x["period_x"]!= before_promo_period or x["period_y"]!= promo_period:
        return set()
    p1 = x["itemsets_x"].split()
    p2 = set(x["itemsets_y"].split())
    if query_name =="adoption":
        return p2.difference(p1)
    if query_name =="loyalty":
        return p2.intersection(p1)
df["adopted_products"] = df.apply(lambda x: promotion_query(x,"adoption"),axis=1)
df["loyalty_products"] = df.apply(lambda x: promotion_query(x,"loyalty"),axis=1)
df.to_csv(output_file)

In [47]:
adoption_counts,loyalty_counts= [],[]
for i,j in zip(df.adopted_products.values.tolist(),df.loyalty_products.values.tolist()):
    if i!=set():
        adoption_counts+= list(i) 
    if j!=set():
        loyalty_counts += list(j)
adoption_counts, loyalty_counts = Counter(adoption_counts),Counter(loyalty_counts)

In [58]:
encoder = LabelEncoder()
encoder.classes_ = np.load(f'../../plots/encoders/{file_name}.npy')
items = pd.read_csv("../../datasets/Total/promo_product_maping.csv",index_col=0)
items.set_index("product_id",inplace=True)

In [59]:
items["adoption_counts"]=0
items["loyalty_counts"]=0
for i in adoption_counts:
    items["adoption_counts"].loc[int(i)] = adoption_counts[i]
for j in loyalty_counts:
    items["loyalty_counts"].loc[int(j)] =loyalty_counts[j]

/home/abdelouahab/3env/local/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [60]:
items

,name,adoption_counts,loyalty_counts
product_id,,,
2432,EVIAN 50CL,90,1
3066,TOTAL ADBLUE 10L,88,9
4219,TOTAL QUARTZ INEO LONG LIFE 5W30 1L,15,0
5333,ADBLUE 10L,36,1
704,CR EVIAN PET 50CL,48,3
4206,TOTAL ADBLUE BIDON 10L,29,2
3039,TOTAL ANTI CREV,1,0
4213,TOTAL ACTIVA 9000 5W40 2L,14,0
2428,EVIAN 50CL PET,0,0


In [51]:
adoption_counts

Counter({'4206': 29,
         '5333': 36,
         '3066': 88,
         '2432': 90,
         '704': 48,
         '4219': 15,
         '5742': 3,
         '4427': 1,
         '4213': 14,
         '3039': 1,
         '6436': 1})